In [1]:
# Import the modules
import cv2
from sklearn.externals import joblib
import numpy as np
import sys, getopt
from keras.models import load_model
    # Load the Keras CNN trained model
model = load_model('cDigit1.h5')
inp_pic = "Digit1.jpg"
    # Original image
im = cv2.imread(inp_pic)
cv2.imshow("Original Image", im)
cv2.waitKey()
cv2.destroyAllWindows()

    # Read image in grayscale mode
img = cv2.imread(inp_pic,0)
    
    # Median Blur and Gaussian Blur to remove Noise
img = cv2.medianBlur(img,3)
img = cv2.GaussianBlur(img, (5, 5), 0)
print(img)

    # Adaptive Threshold for handling lightning
im_th = cv2.adaptiveThreshold(img,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY_INV,11,5)
print(im_th)
    # cv2.imshow("Threshold Image",im_th)
kernel = np.ones((1,1),np.uint8) 
im_th = cv2.dilate(im_th,kernel,iterations = 4)
print(im_th)
cv2.imshow("After", im_th)
cv2.destroyAllWindows()
    ##################################################################################################


    # Find contours in the image
ctrs, hier, _ = cv2.findContours(im_th.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Get rectangles contains each contour
rects = [cv2.boundingRect(ctr) for ctr in ctrs]
    # For each rectangular region, predict using cnn model
for rect in rects:
        # Draw the rectangles
    cv2.rectangle(im, (rect[0], rect[1]), (rect[0] + rect[2], rect[1] + rect[3]), (0, 255, 0), 3) 
        # Make the rectangular region around the digit
    leng = int(rect[3] * 1.6)
    pt1 = int(rect[1] + rect[3] // 2 - leng // 2)
    pt2 = int(rect[0] + rect[2] // 2 - leng // 2)
    roi = im_th[pt1:pt1+leng, pt2:pt2+leng]
        # Resize the image
    roi = cv2.resize(roi, (28, 28), interpolation=cv2.INTER_AREA)
        
        # Input for CNN Model
    roi = roi[np.newaxis,np.newaxis,:,:]

        # Input for Feed Forward Model
        # roi = roi.flatten()
        # roi = roi[np.newaxis]
    nbr = model.predict_classes(roi,verbose=0)
    cv2.putText(im, str(int(nbr[0])), (int(rect[0]), int(rect[1])),cv2.FONT_HERSHEY_DUPLEX, 2, (0, 255, 255), 3)

cv2.imshow("Resulting Image with Predicted numbers", im)
cv2.waitKey()

if __name__ == "__main__":
   main(sys.argv[1:])

Using TensorFlow backend.
/usr/local/lib/python3.5/dist-packages/keras/engine/topology.py:1242: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(activation="linear", strides=[1, 1], use_bias=True, filters=16, padding="valid", activity_regularizer=None, kernel_size=(3, 3), bias_constraint=None, data_format="channels_first", name="convolution2d_13", input_dtype="float32", kernel_initializer="glorot_uniform", kernel_regularizer=None, bias_regularizer=None, kernel_constraint=None, batch_input_shape=[None, 1, ..., trainable=True)`
  return cls(**config)
/usr/local/lib/python3.5/dist-packages/keras/engine/topology.py:1242: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(activation="linear", strides=[1, 1], use_bias=True, filters=16, activity_regularizer=None, kernel_size=(3, 3), bias_constraint=None, data_format="channels_first", padding="valid", name="convolution2d_14", trainable=True, kernel_initializer="glorot_uniform", kernel_regularizer=None, bias_re

[[125 125 126 ..., 129 129 129]
 [125 125 126 ..., 128 128 128]
 [125 125 127 ..., 128 128 128]
 ..., 
 [ 97  97  97 ..., 104 105 106]
 [ 98  98  98 ..., 104 105 105]
 [ 98  98  98 ..., 104 105 105]]
[[0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 ..., 
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]]
[[0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 ..., 
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]]


error: /home/tanmay-edgelord/OpenCV/modules/imgproc/src/imgwarp.cpp:3492: error: (-215) ssize.width > 0 && ssize.height > 0 in function resize
